In [1]:
%reload_ext autoreload
%autoreload 2

In [18]:
import os, sys
import re
import json
import glob
import datetime
from collections import Counter

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from wordcloud import WordCloud
import yake

In [3]:
os.chdir('..')

In [4]:
from src.loader import NewsDataLoader

In [5]:
from src.config import cfg

Output File: news_output.csv
Path: data
News Data: rating.csv
Traffic Data: trafiic.csv
Domain Location Data: domains_location.csv


In [6]:
import src.utils as utils

In [21]:
news_data_loader = NewsDataLoader(cfg.path)
kw_extractor = yake.KeywordExtractor()

In [8]:
original_news_df = news_data_loader.get_news_data()

In [10]:
# check columns
original_news_df.columns

Index(['article_id', 'source_id', 'source_name', 'author', 'title',
       'description', 'url', 'url_to_image', 'published_at', 'content',
       'category', 'article', 'title_sentiment'],
      dtype='object')

In [12]:
original_news_df.isna().sum()

article_id             0
source_id          40585
source_name            0
author              2163
title                  0
description           10
url                    0
url_to_image        3451
published_at           0
content                0
category              21
article                0
title_sentiment        0
dtype: int64

In [19]:
test_content = original_news_df.head(2)['content'][1]

In [25]:

keywords = kw_extractor.extract_keywords(test_content)

for kw in keywords:
    print(kw)

('Hicham Boudaoui goal', 0.002511861746285796)
('second-half Hicham Boudaoui', 0.0025118617462857964)
('lowly Clermont Foot', 0.0025118617462857964)
('Foot on Friday', 0.0072924691533075596)
('Hicham Boudaoui', 0.010648804518731927)
('Clermont Foot', 0.010648804518731927)
('second-half Hicham', 0.023744920762562488)
('Boudaoui goal', 0.023744920762562488)
('lowly Clermont', 0.023744920762562488)
('moved into provisional', 0.053804342030416036)
('provisional first place', 0.053804342030416036)
('goal that secured', 0.053804342030416036)
('win at lowly', 0.053804342030416036)
('Ligue', 0.07052916133628977)
('Friday', 0.07052916133628977)
('Nice moved', 0.07742423445237898)
('Hicham', 0.10264795322139568)
('Boudaoui', 0.10264795322139568)
('Clermont', 0.10264795322139568)
('Foot', 0.10264795322139568)
